# Wrangle: [testfit.io](https://blog.testfit.io/)

In [132]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle

import itertools

import preprocessing

In [36]:
pd.set_option("display.max_columns", None)
plt.rc("figure", figsize=(16, 8))

In [179]:
df = wrangle.wrangle_hud()

In [128]:
(df.fiscal_year_of_firm_commitment_activity != df.fiscal_year_of_firm_commitment).sum()

0

In [39]:
# encoding activity description
def drop_and_encode(df):
    """one hot encoding activity description"""
    df = pd.get_dummies(data=df, columns=['activity_description'])
    df = df.drop(columns= ['fha_number', 'project_name', 'project_state',
        'basic_fha_risk_share_or_other', 'program_category', 'activity_group',
       'facility_type', 'program_designation', 'firm_commitment_activity',
       'lender_at_firm_commitment_activity',
       'mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance',
       'unit_or_bed_count', 'date_of_firm_commitment_activity',
       'mortgage_at_firm_commitment_issuance', 'date_of_firm_issue',
       'fiscal_year_of_firm_commitment', 'map_or_tap', 'lihtc_designation',
       'tax_exempt_bond_designation', 'current_status'] )
    return df

# 'home_designation', 'cdbg_designation',
#        'section_202_refi_designation', 'irp_decoupling_designation',
#        'hope_vi_designation', 

In [40]:
df = drop_and_encode(df)
df.head()

,project_city,fiscal_year_of_firm_commitment_activity,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,final_mortgage_amount,activity_description_Additions,activity_description_Improvements,activity_description_New Construction,activity_description_Operating Loss,activity_description_Purchase,activity_description_Refinance,activity_description_SubRehab
0,Olney,2006,False,False,False,False,False,271700,0,0,0,0,0,1,0
1,Breckenridge,2006,False,False,False,False,False,168300,0,0,0,0,0,1,0
2,Rocky River,2006,False,False,False,False,False,3886800,0,0,0,0,0,1,0
3,Wilmington,2006,False,False,False,False,False,4000000,0,0,0,0,0,1,0
4,Joliet,2006,False,False,False,False,False,5247700,0,0,0,0,0,1,0


In [52]:
city_mask = df.project_city.value_counts().nlargest(35)

In [53]:
def in_city_mask(x):
    return x in city_mask

In [54]:
pre_df = df[df.project_city.apply(in_city_mask)]

In [88]:
def preprocessing_for_modeling(df):
    """function to manipulate df into df usable for modeling"""
    
    # city mask identifies 35 cities with most hud loans
    city_mask = df.project_city.value_counts().nlargest(35)
    
    
    # apply city mask to shrink the df
    def in_city_mask(x):
        return x in city_mask
    df = df[df.project_city.apply(in_city_mask)]
    
    #create a df for modelling that groups-by year and city aggregating mortgage amount by count, median, mean, and sum
    df_for_model = (df.groupby(['fiscal_year_of_firm_commitment_activity', 'project_city'])
         .final_mortgage_amount.agg(['count', 'median', 'mean', 'sum']).reset_index())
    
    df_for_model = df_for_model.rename(columns={'fiscal_year_of_firm_commitment_activity': 'year', 'project_city': 'city'})
    
    return df_for_model
    

In [89]:
df = preprocessing_for_modeling(df)

In [96]:
df['city_year'] = df.city + "_" + df.year.astype(str)

In [100]:
chicago =  df[df.city == "Chicago"]

In [120]:
chicago['y/y'] = (chicago['count'].diff(1) / chicago['count'])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [119]:
chicago

,year,city,count,median,mean,sum,city_year,y/y
7,2006,Chicago,21,6812900.0,8.188114e+06,171950400,Chicago_2006,NaN
41,2007,Chicago,26,3460850.0,4.438900e+06,115411400,Chicago_2007,NaN
73,2008,Chicago,10,10061200.0,1.061864e+07,106186400,Chicago_2008,-1.100000
105,2009,Chicago,19,11868600.0,1.164747e+07,221301900,Chicago_2009,-0.368421
137,2010,Chicago,23,7250000.0,9.042609e+06,207980000,Chicago_2010,0.565217
172,2011,Chicago,43,8677500.0,1.148943e+07,494045600,Chicago_2011,0.558140
206,2012,Chicago,45,8300000.0,1.156243e+07,520309300,Chicago_2012,0.488889
241,2013,Chicago,46,9517950.0,1.068399e+07,491463400,Chicago_2013,0.065217
276,2014,Chicago,20,3556100.0,1.074232e+07,214846300,Chicago_2014,-1.250000
311,2015,Chicago,20,6769000.0,1.044247e+07,208849400,Chicago_2015,-1.300000


In [ ]:
refinance_mask = df.activity_description_Refinance == 1

In [65]:
(df[refinance_mask].groupby(['fiscal_year_of_firm_commitment_activity', 'project_city'])
         .final_mortgage_amount.agg(['count', 'median', 'mean', 'sum']).reset_index())

,fiscal_year_of_firm_commitment_activity,project_city,count,median,mean,sum
0,2006,Aiken,1,2447400.0,2447400.0,2447400
1,2006,Akron,1,2480000.0,2480000.0,2480000
2,2006,Albany,1,1002600.0,1002600.0,1002600
3,2006,Alexandria,1,1221400.0,1221400.0,1221400
4,2006,Algood,2,646000.0,646000.0,1292000
...,...,...,...,...,...,...
10379,2020,Woodland Park,1,32000000.0,32000000.0,32000000
10380,2020,ZANESVILLE,1,12030100.0,12030100.0,12030100
10381,2020,Zion,1,8293200.0,8293200.0,8293200
10382,2020,haskell,1,2504000.0,2504000.0,2504000


# Explore Object Columns

In [ ]:
object_columns = []
for col in df.columns:
    if df[col].dtype == 'object':
        object_columns.append(col)
object_columns

In [203]:
#fha_number
repeat_numbers = df.shape[0] - len(df.fha_number.unique())
repeat_numbers

516

In [206]:
def in_repeat_list(x):
    return x in repeat_list

In [207]:
df_repeat = df[df.fha_number.apply(in_repeat_list)].sort_values('fha_number')

In [208]:
df_repeat.firm_commitment_activity.value_counts()

Firm Reissued    511
Firm Issued      508
Name: firm_commitment_activity, dtype: int64

In [209]:
df.firm_commitment_activity.value_counts()

Firm Issued      20381
Firm Reissued      537
Name: firm_commitment_activity, dtype: int64

In [210]:
#project_name
repeat_name = df.shape[0] - len(df.project_name.unique())
repeat_name

2551

In [220]:
fha_numbers_max = df_repeat.groupby('fha_number').date_of_firm_commitment_activity.idxmax()

- take dataframe and find multiple fha_numbers
- get the most recent row of the repeat fha_numbers

    - make df of only uniques
    - make df of max from repeat fha_numbers
    - concat two dfs

In [228]:
fha_value_counts = df.fha_number.value_counts()
repeat_list = list(fha_value_counts[fha_value_counts != 1].index)

In [264]:
# list of unique fha numbers
unique_fha_numbers = list(df.fha_number.value_counts()[df.fha_number.value_counts() == 1]

SyntaxError: unexpected EOF while parsing (<ipython-input-264-410e0f051f62>, line 2)

In [272]:
df.groupby('fha_number').date_of_firm_commitment_activity.max()

fha_number
10010      2007-12-21
10011      2008-02-28
10012      2016-04-20
11142      2014-08-08
11183      2008-11-25
              ...    
17622002   2009-05-14
17622003   2013-07-09
17622004   2015-04-01
17635048   2011-01-21
17635049   2011-09-30
Name: date_of_firm_commitment_activity, Length: 20402, dtype: datetime64[ns]

In [281]:
df_repeat.groupby('fha_number').date_of_firm_commitment_activity.max()

fha_number
35493      2012-07-31
35512      2013-12-04
35519      2014-07-24
38016      2014-05-15
43077      2018-08-22
              ...    
14311096   2011-08-22
14311100   2011-02-24
14311110   2011-12-27
14311174   2015-12-15
14322057   2011-04-15
Name: date_of_firm_commitment_activity, Length: 503, dtype: datetime64[ns]

In [276]:
repeat_fha_numbers

[Timestamp('2012-07-31 00:00:00'),
 Timestamp('2013-12-04 00:00:00'),
 Timestamp('2014-07-24 00:00:00'),
 Timestamp('2014-05-15 00:00:00'),
 Timestamp('2018-08-22 00:00:00'),
 Timestamp('2016-11-30 00:00:00'),
 Timestamp('2007-03-06 00:00:00'),
 Timestamp('2009-04-17 00:00:00'),
 Timestamp('2006-05-30 00:00:00'),
 Timestamp('2016-07-29 00:00:00'),
 Timestamp('2007-02-15 00:00:00'),
 Timestamp('2007-07-03 00:00:00'),
 Timestamp('2013-12-11 00:00:00'),
 Timestamp('2015-10-21 00:00:00'),
 Timestamp('2006-09-07 00:00:00'),
 Timestamp('2008-04-25 00:00:00'),
 Timestamp('2007-12-28 00:00:00'),
 Timestamp('2009-05-20 00:00:00'),
 Timestamp('2013-08-29 00:00:00'),
 Timestamp('2009-12-07 00:00:00'),
 Timestamp('2012-03-14 00:00:00'),
 Timestamp('2016-04-01 00:00:00'),
 Timestamp('2012-03-15 00:00:00'),
 Timestamp('2013-11-26 00:00:00'),
 Timestamp('2008-06-18 00:00:00'),
 Timestamp('2014-08-02 00:00:00'),
 Timestamp('2018-09-14 00:00:00'),
 Timestamp('2007-07-24 00:00:00'),
 Timestamp('2012-02-

In [259]:
len(all_unique_fha_numbers)

20402

In [269]:
df.columns

Index(['fha_number', 'project_name', 'project_city', 'project_state',
       'basic_fha_risk_share_or_other', 'program_category',
       'activity_description', 'activity_group', 'facility_type',
       'program_designation', 'firm_commitment_activity',
       'lender_at_firm_commitment_activity',
       'mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance',
       'unit_or_bed_count', 'date_of_firm_commitment_activity',
       'fiscal_year_of_firm_commitment_activity',
       'mortgage_at_firm_commitment_issuance', 'date_of_firm_issue',
       'fiscal_year_of_firm_commitment', 'map_or_tap', 'lihtc_designation',
       'tax_exempt_bond_designation', 'home_designation', 'cdbg_designation',
       'section_202_refi_designation', 'irp_decoupling_designation',
       'hope_vi_designation', 'current_status', 'final_mortgage_amount'],
      dtype='object')

In [249]:
len(all_unique_fha_numbers)

20402

In [252]:
def in_unique_list(x):
    return x in all_unique_fha_numbers

In [261]:
df.fha_number.apply(in_unique_list).mean()

1.0

,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
0,11345004,Western Heights Apartments,Olney,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,277600,48,2005-10-01,2006,277600,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Firm Comit then Owner Withdrew,271700
1,11345005,Woodlands Village Apts,Breckenridge,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,173000,88,2005-10-01,2006,173000,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,168300
2,4235583,Harbor Court Apartments,Rocky River,OH,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,GREYSTONE SERVICING COMPANY LLC,3886800,120,2005-10-03,2006,3886800,2005-10-03,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,3886800
3,3297260,Compton Towne Apartments,Wilmington,DE,Risk Sharing,QPE Risk Sharing,Refinance,Refi/ Pchse,Coop/Apts/Mobile Home Pk/ALF,QPE Risk Shg Existing,Firm Issued,FEDERAL NATIONAL MORTGAGE,4000000,76,2005-10-03,2006,4000000,2005-10-03,2006,TAP,True,False,False,False,False,False,False,Finally Endorsed,4000000
4,7135779,Evergreen Terrace I,Joliet,IL,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,REILLY MORTGAGE GRP,5129000,241,2005-10-04,2006,5129000,2005-10-04,2006,TAP,False,False,False,False,False,False,False,Firm Expired,5247700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20913,12711269,Quail Run Apartments,Vancouver,WA,Basic FHA,223f Refi/Purchase Apts,Refinance,Refi/ Pchse,Apts/Coops,223f Apts,Firm Issued,SUNTRUST BANK,17051600,129,2020-03-31,2020,17051600,2020-03-31,2020,MAP,True,False,False,False,False,False,False,Firm Issued,17051600
20914,2210003,Legacy Memory Care,Falmouth,ME,Basic FHA,232 Health Care,Additions,Impv/Adds,Asst'd Livg,ALF 241a,Firm Issued,PRUDENTIAL HUNTOON PAIGE,1189200,30,2020-03-31,2020,1189200,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,1189200
20915,6722256,Consulate Health Care of Kissimmee,Kissimmee,FL,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,ORIX REAL ESTATE CAPITAL LLC,17920000,120,2020-03-31,2020,17920000,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,17920000
20916,11322284,The Villages of Lake Highlands,Dallas,TX,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,ORIX REAL ESTATE CAPITAL LLC,20756500,154,2020-03-31,2020,20756500,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,20756500


In [212]:
len(df_repeat.final_mortgage_amount.unique())

489

for fha_number drop if not max date

In [ ]:
df.basic_fha_risk_share_or_other.value_counts()

In [ ]:
df.groupby('basic_fha_risk_share_or_other').final_mortgage_amount.mean().plot.bar()

In [ ]:
for col in object_columns:
    print(col)
    print(df[col].value_counts())
    print()

In [ ]:
df[df.project_state == 'TX'].project_city.value_counts()

In [ ]:
df.activity_description.value_counts().plot.barh()

In [ ]:
df.current_status.value_counts()

In [ ]:
df.activity_description.value_counts()

In [ ]:
df.groupby('map_or_tap').final_mortgage_amount.count()

In [ ]:
df.groupby('facility_type').final_mortgage_amount.count()

In [ ]:
df.head()

In [ ]:

df[.groupby('project_city').final_mortgage_amount.mean()

In [ ]:
city_mask = df.project_city.value_counts().nlargest(50).index

In [ ]:
city_mask

In [133]:
df_model = preprocessing.get_model_df()

In [134]:
df_model

,city,state,year,activity_description,quantity_of_mortgages,total_mortgage_amount,average_mortgage_amount,median_mortgage_amount,label
0,Bronx,NY,2013,Refinance,8,822430600,1.028038e+08,24774500.0,False
1,Trenton,NJ,2009,New Construction,1,755874500,7.558745e+08,755874500.0,False
2,New York,NY,2013,Improvements,1,500000000,5.000000e+08,500000000.0,False
3,Chicago,IL,2012,Refinance,41,476952400,1.163299e+07,8300000.0,False
4,Chicago,IL,2013,Refinance,42,428192800,1.019507e+07,9517950.0,False
...,...,...,...,...,...,...,...,...,...
15251,Cordele,GA,2008,Refinance,1,145000,1.450000e+05,145000.0,False
15252,Covington,KY,2006,SubRehab,1,125000,1.250000e+05,125000.0,False
15253,Wolcottville,IN,2017,Refinance,1,114000,1.140000e+05,114000.0,False
15254,Aurora,NC,2007,Refinance,1,70100,7.010000e+04,70100.0,False


In [135]:
df

,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
0,11345004,Western Heights Apartments,Olney,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,277600,48,2005-10-01,2006,277600,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Firm Comit then Owner Withdrew,271700
1,11345005,Woodlands Village Apts,Breckenridge,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,173000,88,2005-10-01,2006,173000,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,168300
2,4235583,Harbor Court Apartments,Rocky River,OH,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,GREYSTONE SERVICING COMPANY LLC,3886800,120,2005-10-03,2006,3886800,2005-10-03,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,3886800
3,3297260,Compton Towne Apartments,Wilmington,DE,Risk Sharing,QPE Risk Sharing,Refinance,Refi/ Pchse,Coop/Apts/Mobile Home Pk/ALF,QPE Risk Shg Existing,Firm Issued,FEDERAL NATIONAL MORTGAGE,4000000,76,2005-10-03,2006,4000000,2005-10-03,2006,TAP,True,False,False,False,False,False,False,Finally Endorsed,4000000
4,7135779,Evergreen Terrace I,Joliet,IL,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,REILLY MORTGAGE GRP,5129000,241,2005-10-04,2006,5129000,2005-10-04,2006,TAP,False,False,False,False,False,False,False,Firm Expired,5247700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20913,12711269,Quail Run Apartments,Vancouver,WA,Basic FHA,223f Refi/Purchase Apts,Refinance,Refi/ Pchse,Apts/Coops,223f Apts,Firm Issued,SUNTRUST BANK,17051600,129,2020-03-31,2020,17051600,2020-03-31,2020,MAP,True,False,False,False,False,False,False,Firm Issued,17051600
20914,2210003,Legacy Memory Care,Falmouth,ME,Basic FHA,232 Health Care,Additions,Impv/Adds,Asst'd Livg,ALF 241a,Firm Issued,PRUDENTIAL HUNTOON PAIGE,1189200,30,2020-03-31,2020,1189200,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,1189200
20915,6722256,Consulate Health Care of Kissimmee,Kissimmee,FL,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,ORIX REAL ESTATE CAPITAL LLC,17920000,120,2020-03-31,2020,17920000,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,17920000
20916,11322284,The Villages of Lake Highlands,Dallas,TX,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,ORIX REAL ESTATE CAPITAL LLC,20756500,154,2020-03-31,2020,20756500,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,20756500


In [136]:
city_mask = df.project_city.value_counts().nlargest(35)

Chicago         346
Indianapolis    259
Columbus        173
Los Angeles     147
Houston         140
Boston          131
San Antonio     126
Baltimore       125
Dallas          113
Atlanta         113
Washington      104
New York         96
Denver           95
Cleveland        90
Portland         89
St. Louis        88
Cincinnati       84
Austin           83
Minneapolis      81
Richmond         78
Phoenix          76
Charlotte        75
Jacksonville     74
Columbia         72
Bronx            70
Memphis          70
Orlando          69
Knoxville        69
Detroit          68
Greenville       62
Fort Worth       61
Milwaukee        61
Brooklyn         60
Madison          59
Toledo           59
Name: project_city, dtype: int64

In [140]:
def in_city_mask(x):
        return x in city_mask
df = df[df.project_city.apply(in_city_mask)]

In [143]:
df.project_city.value_counts()

Chicago         346
Indianapolis    259
Columbus        173
Los Angeles     147
Houston         140
Boston          131
San Antonio     126
Baltimore       125
Dallas          113
Atlanta         113
Washington      104
New York         96
Denver           95
Cleveland        90
Portland         89
St. Louis        88
Cincinnati       84
Austin           83
Minneapolis      81
Richmond         78
Phoenix          76
Charlotte        75
Jacksonville     74
Columbia         72
Memphis          70
Bronx            70
Knoxville        69
Orlando          69
Detroit          68
Greenville       62
Milwaukee        61
Fort Worth       61
Brooklyn         60
Madison          59
Toledo           59
Name: project_city, dtype: int64

In [200]:
def preprocessing_for_modeling(df):
    """function to manipulate df into df usable for modeling"""
    
    # city mask identifies 35 cities with most hud loans
    city_mask = df.project_city.value_counts().nlargest(35)
    
    
    # apply city mask to shrink the df
    def in_city_mask(x):
        return x in city_mask
    df = df[df.project_city.apply(in_city_mask)]
    
    #create a df for modelling that groups-by year and city aggregating mortgage amount by count, median, mean, and sum
    df_for_model = (df.groupby(['fiscal_year_of_firm_commitment_activity', 'project_city'])
         .final_mortgage_amount.agg(['count', 'median', 'mean', 'sum']).reset_index())
    
    #  only take cities that have an observation for every year
    observations_mask =  df_for_model.project_city.value_counts()[df_for_model.project_city.value_counts() == 15]
    
     # apply city mask to shrink the df
    def in_observations_mask(x):
        return x in observations_mask
    df_for_model = df_for_model[df_for_model.project_city.apply(in_observations_mask)]
    
    
    
    df_for_model = df_for_model.rename(columns={'fiscal_year_of_firm_commitment_activity': 'year', 'project_city': 'city'})
    
    return df_for_model

In [201]:
df_prep = preprocessing_for_modeling(df)

In [202]:
df_prep

,year,city,count,median,mean,sum
0,2006,Atlanta,6,4457100.0,5.531133e+06,33186800
1,2006,Austin,1,14083843.0,1.408384e+07,14083843
2,2006,Baltimore,6,4570450.0,5.993167e+06,35959000
3,2006,Boston,7,3941300.0,7.059614e+06,49417300
7,2006,Chicago,21,6812900.0,8.188114e+06,171950400
...,...,...,...,...,...,...
502,2020,Orlando,1,16604000.0,1.660400e+07,16604000
503,2020,Phoenix,3,30000000.0,2.624000e+07,78720000
504,2020,Portland,4,11486550.0,1.150535e+07,46021400
507,2020,St. Louis,3,6316200.0,7.585433e+06,22756300


,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
0,11345004,Western Heights Apartments,Olney,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,277600,48,2005-10-01,2006,277600,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Firm Comit then Owner Withdrew,271700
1,11345005,Woodlands Village Apts,Breckenridge,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,173000,88,2005-10-01,2006,173000,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,168300
2,4235583,Harbor Court Apartments,Rocky River,OH,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,GREYSTONE SERVICING COMPANY LLC,3886800,120,2005-10-03,2006,3886800,2005-10-03,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,3886800
3,3297260,Compton Towne Apartments,Wilmington,DE,Risk Sharing,QPE Risk Sharing,Refinance,Refi/ Pchse,Coop/Apts/Mobile Home Pk/ALF,QPE Risk Shg Existing,Firm Issued,FEDERAL NATIONAL MORTGAGE,4000000,76,2005-10-03,2006,4000000,2005-10-03,2006,TAP,True,False,False,False,False,False,False,Finally Endorsed,4000000
4,7135779,Evergreen Terrace I,Joliet,IL,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,REILLY MORTGAGE GRP,5129000,241,2005-10-04,2006,5129000,2005-10-04,2006,TAP,False,False,False,False,False,False,False,Firm Expired,5247700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20913,12711269,Quail Run Apartments,Vancouver,WA,Basic FHA,223f Refi/Purchase Apts,Refinance,Refi/ Pchse,Apts/Coops,223f Apts,Firm Issued,SUNTRUST BANK,17051600,129,2020-03-31,2020,17051600,2020-03-31,2020,MAP,True,False,False,False,False,False,False,Firm Issued,17051600
20914,2210003,Legacy Memory Care,Falmouth,ME,Basic FHA,232 Health Care,Additions,Impv/Adds,Asst'd Livg,ALF 241a,Firm Issued,PRUDENTIAL HUNTOON PAIGE,1189200,30,2020-03-31,2020,1189200,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,1189200
20915,6722256,Consulate Health Care of Kissimmee,Kissimmee,FL,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,ORIX REAL ESTATE CAPITAL LLC,17920000,120,2020-03-31,2020,17920000,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,17920000
20916,11322284,The Villages of Lake Highlands,Dallas,TX,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,ORIX REAL ESTATE CAPITAL LLC,20756500,154,2020-03-31,2020,20756500,2020-03-31,2020,TAP,False,False,False,False,False,False,False,Firm Issued,20756500


In [190]:
city_mask = df.project_city.value_counts().nlargest(35)

In [191]:
city_mask

Chicago         346
Indianapolis    259
Columbus        173
Los Angeles     147
Houston         140
Boston          131
San Antonio     126
Baltimore       125
Dallas          113
Atlanta         113
Washington      104
New York         96
Denver           95
Cleveland        90
Portland         89
St. Louis        88
Cincinnati       84
Austin           83
Minneapolis      81
Richmond         78
Phoenix          76
Charlotte        75
Jacksonville     74
Columbia         72
Bronx            70
Memphis          70
Orlando          69
Knoxville        69
Detroit          68
Greenville       62
Fort Worth       61
Milwaukee        61
Brooklyn         60
Madison          59
Toledo           59
Name: project_city, dtype: int64

In [192]:
def in_city_mask(x):
    return x in city_mask
df2 = df[df.project_city.apply(in_city_mask)]

In [193]:
df2

,fha_number,project_name,project_city,project_state,basic_fha_risk_share_or_other,program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,mortgage_at_time_of_firm_commitment_issuance_amendment_or_reissuance,unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
12,7335668,Creekside Square II Apartments,Indianapolis,IN,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,MERCHANTS CAPITAL CORP,2939300,64,2005-10-06,2006,2939300,2005-10-06,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,2939300
23,3322032,Humbert Lane Health Care Centre,Washington,PA,Basic FHA,232 Health Care,Refinance,Refi/ Pchse,Nursing/ICF,Nursing/ ICF 223f,Firm Issued,SECORE FINANCIAL CORP,6059200,120,2005-10-11,2006,6059200,2005-10-11,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,6059200
29,6135628,Veranda at the Villages at Carver,Atlanta,GA,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,AMERICAN PROPERTY FINCG,3988000,90,2005-10-14,2006,3988000,2005-10-14,2006,MAP,True,False,False,False,False,False,False,Finally Endorsed,3988000
37,7335667,Autumn Chase Apartments,Indianapolis,IN,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,MERCHANTS CAPITAL CORP,10536000,176,2005-10-17,2006,10536000,2005-10-17,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,10536000
38,4211082,Triangle Apartments,Cleveland,OH,Basic FHA,223f Refi/Purchase Apts,Purchase,Refi/ Pchse,Apts/Coops,223f Apts,Firm Issued,ROSE COMMUNITY CAPITAL,17637400,273,2005-10-17,2006,17637400,2005-10-17,2006,MAP,False,False,False,False,False,False,False,Finally Endorsed,17637400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20888,2398425,Mattapan Station 9%,Boston,MA,Risk Sharing,HFA Risk Sharing,New Construction,NC/SR,Coop/Apts/Mobile Home Pk/ALF,HFA Risk Shg NC/SR,Firm Issued,MASSACHUSETTS HFA,2140000,21,2020-03-24,2020,2140000,2020-03-24,2020,TAP,True,False,False,False,False,False,False,Firm Amended,2140000
20890,2398432,Whittier Phase 2,Boston,MA,Risk Sharing,HFA Risk Sharing,New Construction,NC/SR,Coop/Apts/Mobile Home Pk/ALF,HFA Risk Shg NC/SR,Firm Issued,MASSACHUSETTS HFA,15731824,52,2020-03-25,2020,15731824,2020-03-25,2020,TAP,True,True,False,False,False,False,False,Firm Amended,15731824
20908,12611184,Rockwood Station,Portland,OR,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/223f,Firm Issued,GERSHMAN INVESTMENT,4295300,195,2020-03-30,2020,4295300,2020-03-30,2020,MAP,False,False,False,False,False,False,False,Firm Issued,4295300
20909,6135913,CITY LIGHTS II-A,Atlanta,GA,Basic FHA,FHA NC/SR Apts,New Construction,NC/SR,Apts,221d4,Firm Issued,PRUDENTIAL HUNTOON PAIGE,21500000,123,2020-03-31,2020,21500000,2020-03-31,2020,MAP,True,False,False,False,False,False,False,Firm Issued,21500000


In [199]:
df2.project_city.value_counts()

Chicago         346
Indianapolis    259
Columbus        173
Los Angeles     147
Houston         140
Boston          131
San Antonio     126
Baltimore       125
Dallas          113
Atlanta         113
Washington      104
New York         96
Denver           95
Cleveland        90
Portland         89
St. Louis        88
Cincinnati       84
Austin           83
Minneapolis      81
Richmond         78
Phoenix          76
Charlotte        75
Jacksonville     74
Columbia         72
Memphis          70
Bronx            70
Knoxville        69
Orlando          69
Detroit          68
Greenville       62
Milwaukee        61
Fort Worth       61
Brooklyn         60
Madison          59
Toledo           59
Name: project_city, dtype: int64

In [196]:
observations_mask

Series([], Name: project_city, dtype: int64)